In [1]:
import os
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from nltk.lm import Vocabulary
import pickle
import sys
import torch
import importlib
from timeit import default_timer as t
sys.path.append("../../lib")
from metrics import levenshtein
import ocr_correction
from pytorch_decoding import seq2seq

In [2]:
folder = "../../data/ICDAR2019_POCR_competition_dataset/ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/SL/"

In [3]:
files = sorted(os.listdir(folder))
len(files)

1

In [4]:
import glob

files = glob.glob(folder + '/**/*.txt', recursive=True)
len(files)

48

In [5]:
data = []
for f in tqdm(files):
    with open(f) as file:
        data.append(file.readlines())

  0%|          | 0/48 [00:00<?, ?it/s]

In [6]:
data = pd.DataFrame(data, 
                    columns = ["ocr_to_input", 
                               "ocr_aligned", 
                               "gs_aligned"])\
.assign(ocr_to_input = lambda df: df.ocr_to_input.str.replace("[OCR_toInput] ", "", regex = False),
        ocr_aligned = lambda df: df.ocr_aligned.str.replace("[OCR_aligned] ", "", regex = False),
        gs_aligned = lambda df: df.gs_aligned.str.replace("[ GS_aligned] ", "", regex = False))

print(data.shape)
data.head()

(48, 3)


,ocr_to_input,ocr_aligned,gs_aligned
0,Kazalo. Fervo poglavje: Božična pesem .... . B...,@Kazalo. Fervo poglavje: Božična pesem .@.@.@....,Kazalo. Pervo poglavje: Božična pesem . . . ....
1,♦v-**** / n Ta tvoiga telefa sad Zhefhen bodi ...,♦v-**** / n Ta tvoiga telefa sad Zhefhen bodi ...,@@@@@@@ @13 Ta tvoiga telesa sad Zheshen bodi ...
2,9» vershajo da sem soblizhiara ra an kathn per...,@9» vershajo da sem soblizhiara ra an kathn pe...,92 vershajo da sem soblizhia@m na an k@smn pe...
3,"13 - učilnic, na ministerski ukaz, dan 1848. l...","@@@13 - učilnic, na ministerski ukaz, dan 1848...","13 učilnic, na ministerski ukaz, dán 1848..."
4,"Chrijtufoviga Tčrpleina. 49 nami, zhes nas v’ ...","@Chrijtufoviga Tčrpleina. 49 nami, zhes nas v’...","Christusoviga Terpleina. 49 nami, zhes nas v'..."


In [7]:
data.applymap(len).describe()

,ocr_to_input,ocr_aligned,gs_aligned
count,48.000000,48.000000,48.000000
mean,2010.062500,2097.875000,2096.875000
std,2006.992748,2150.721548,2150.721548
min,339.000000,345.000000,344.000000
25%,811.500000,841.500000,840.500000
50%,1360.000000,1417.500000,1416.500000
75%,1617.750000,1681.500000,1680.500000
max,7062.000000,8289.000000,8288.000000


In [8]:
vocabulary = Vocabulary(data.ocr_to_input.sum() + data.ocr_aligned.sum() + data.gs_aligned.sum())
print(len(vocabulary))

144


In [9]:
distances = levenshtein(reference = data.gs_aligned.str.replace("@", ""), 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count       48.000000
mean       344.811294
std       1940.190506
min          1.466049
25%          6.338192
50%         10.309256
75%         15.433858
max      13250.000000
Name: cer, dtype: float64

In [10]:
distances = levenshtein(reference = data.gs_aligned, 
                        hypothesis = data.ocr_to_input)
distances.cer.describe()

count    48.000000
mean     16.417297
std      20.757412
min       1.462664
25%       6.264535
50%      10.189735
75%      14.902369
max      99.437148
Name: cer, dtype: float64

In [11]:
data.to_pickle("../../data/sl/data/test.pkl")